In [ ]:
import random
import numpy as np

In [ ]:
import pandas as pd
from random import randint
!pip install transformers
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

     |████████████████████████████████| 675kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 14.0MB/s 
     |████████████████████████████████| 3.8MB 13.5MB/s 
     |████████████████████████████████| 890kB 42.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=0febeab0d0ddf891fdbe38e70845ed463945e2f26099311ef7e56c9e15333774
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


**Defining the entity (column name/variable) and phrase classes**


---


Class defines what semantic annotation it generates

In [ ]:
# Quick and dirty implementation
class column_name:
  def __init__(self,name):
    self.name = name
  def return_dic(self):
    return {self.name: self}
  def generate(self,source):
    #set and return the column name
    self.column = source.columns[randint(0, len(source.columns)-1)]
    self.value = str(self.column)
    return self.value

class var:
  def __init__(self,name): #paired column object
    self.name = name
  def attach_column(self,column_name):
    self.attached_column = column_name
    return self
  def attach_return(self,column_name):
    self.attached_column = column_name
    return {column_name.name: column_name, self.name: self}  
  def generate(self, source): #get the source and column name from the paired column object 
    # get the series by the column name
    self.value = str(source[self.attached_column.column].iloc[randint(0, len(source)-1)])
    return self.value

# ToDo: add components element here (a (list) of component selectors )
class recombinator:
  def __init__(self,representation,variations,inputs,sources,component_selectors=[]): #representation stays the same, variations contain variables and colums names which are taken from the source 
    self.representation = representation #fixed
    self.variations = variations #list
    self.inputs = inputs
    self.sources = sources #for each variation several sources are used
    self.component_selectors = component_selectors
    # generate the dict
    self.input_dic = {}
    for input in inputs:
      if isinstance(input,tuple):
        self.input_dic[input[0].name] = input[0]
        self.input_dic[input[1].name] = input[1].attach_column(input[0])
      else:
        self.input_dic[input.name] = input

# ToDo: add components feature here 
  def generate(self, n): #n is the number of cycles within the nested loop
    gen_texts = []
    for variation in self.variations:
      for source in self.sources:
          for i in range(n):
            #get the components here 
            # get the length of inputs 
            inp_n = len(self.inputs)
            #for each selector embed components (random # of them):
            temp_input_dic = self.input_dic
            temp_representation = self.representation
            temp_variation = variation
            for (selector,anchor,max_repeat) in self.component_selectors:
              reps = np.random.randint(0,max_repeat)
              for rep in range(reps):
                embed_parse = selector.pick(0).get_parse()(inp_n)
                embed_text = selector.stored[0].get_text()(inp_n)
                temp_input_dic = {**temp_input_dic,**selector.stored[0].inputs(inp_n)}
                ins_rep_i = temp_representation.index(anchor)
                ins_var_i = temp_variation.index(anchor)
                temp_representation = temp_representation[:ins_rep_i] + ' ' + embed_parse + temp_representation[ins_rep_i:]
                temp_variation = temp_variation[:ins_var_i] + ' ' + embed_text + temp_variation[ins_var_i:]
                inp_n += 1
              # > remove anchor
              temp_representation = temp_representation.replace(anchor,'')
              temp_variation = temp_variation.replace(anchor,'')
            #instantiate the variables by calling the generate function
            gen_dic = {}
            ner_dic = {}
            for k in temp_input_dic:
              gen_dic[k] = temp_input_dic[k].generate(source)
              #tokenize the obtained text
              token_list = tokenizer.tokenize(gen_dic[k])
              ner_dic[k] = ' '.join([k]*len(token_list)) # replicated the class
            variation_text = temp_variation.format(**gen_dic) #nice trick here
            ner_text = temp_variation.format(**ner_dic) #embed the replicated ner class
            ner_tokens = ner_text.split(' ')
            ner_target = []
            for token in ner_tokens:
              if token in list(temp_input_dic.keys()):
                ner_target.append(token)
              else:
                token_list = tokenizer.tokenize(token)
                ner_target = ner_target + ['O']*len(token_list)
            #print(tokenizer.tokenize(variation_text),ner_target)
            gen_texts.append((variation_text,temp_representation,' '.join(ner_target))) 
            #ner_list[i_s:i_e+1] = [k]*len(ner_list[i_s:i_e+1]) #REPLACEMENT TRICK     
            #representation_text = self.representation.format(**rep_dic)
            #ext_ner_list[i+ext_num:i+ext_num] = [ner_list[i]]*(len(split)-1) #INCERTION TRICK
    return pd.DataFrame(gen_texts,columns=['text','rep','ner'])

In [ ]:
# add componnets list to recombinator, use * for insertion, set the last var and col indexes by len(inputs)

In [ ]:
# New class 'components' - allows to embed and combine sentence parts
class component:
  def __init__(self,parse,text_list,inputs):
    self.parse = parse
    self.text_list = text_list
    self.inputs = inputs

  def get_parse(self):
    return self.parse
  def get_text(self):
    return random.choice(self.text_list)

class component_selector:
  def __init__(self,component_list):
    self.component_list = component_list
    self.stored = {}
  def pick(self,i):
    self.stored[i] = random.choice(self.component_list)
    return self.stored[i]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
zoo_df= pd.read_csv('/content/drive/My Drive/Datasets/zoo2.csv')
zoo3_df = pd.read_csv('/content/drive/My Drive/Datasets/zoo3.csv')
avocado_df = pd.read_csv('/content/drive/My Drive/Datasets/avocado.csv')
cars_df = pd.read_csv('/content/drive/My Drive/Datasets/cars.csv')
wildfires_df = pd.read_csv('/content/drive/My Drive/Datasets/wildfires.csv')
bike_df = pd.read_csv('/content/drive/My Drive/Datasets/bike.csv')

In [ ]:
# Components 
# ADD MORE VARIATION HERE! {col_3} being {var_3} / which have {var_2} {col_2} / having {var_2} {col_2}
eq_comp = component(parse = lambda i: f'f_eq col_{i+1} var_{i+1}',text_list=[lambda i: f'{{var_{i+1}}} {{col_{i+1}}}',
                                                                        lambda i: f'{{col_{i+1}}} is {{var_{i+1}}}',
                                                                        lambda i: f'{{col_{i+1}}} is equal to {{var_{i+1}}}',
                                                                        lambda i: f'{{col_{i+1}}} being {{var_{i+1}}}',
                                                                        lambda i: f'{{col_{i+1}}} being equal {{var_{i+1}}}',
                                                                        lambda i: f'having {{var_{i+1}}} {{col_{i+1}}}'],
                    inputs = lambda i: var(f'var_{i+1}').attach_return(column_name(f'col_{i+1}'))) #NOTICE the attach_return funtion!
cond_components = component_selector([eq_comp])
and_eq = component(parse = lambda i: f'f_and {cond_components.pick(0).get_parse()(i)}',
                     text_list = [lambda i: f'and {cond_components.stored[0].get_text()(i)}'],
                     inputs = lambda i: var(f'var_{i+1}').attach_return(column_name(f'col_{i+1}')))
or_eq = component(parse = lambda i: f'f_or {cond_components.pick(0).get_parse()(i)}',
                     text_list = [lambda i: f'or {cond_components.stored[0].get_text()(i)}'],
                     inputs = lambda i: var(f'var_{i+1}').attach_return(column_name(f'col_{i+1}')))
logic_components = component_selector([and_eq,or_eq])
### Q: Do we actually want to use 'f_and' here? 
and_col = component(parse = lambda i: f'f_and col_{i+1}', text_list = [lambda i: f'and {{col_{i+1}}}'], inputs = lambda i: column_name(f'col_{i+1}').return_dic())
and_col_selector = component_selector([and_col])

In [ ]:
print(and_col.get_parse()(1))
print(and_col.get_text()(1))

f_and col_2
and {col_2}


In [ ]:
# always appended to existing condition 
# the separate AND and OR recombinators are not necessary
print(logic_components.pick(0).get_parse()(1))
print(logic_components.stored[0].get_text()(1))

f_and f_eq col_2 var_2
and {col_2} being {var_2}


In [ ]:
# for sample 1 add a variation with several selected attributes
# Important! Add the samples where var is set to True(1 or >0 ) by the engine (e.g. Pick the animals which have a tail, which have legs, think about the 'do not' sample )
#start s_name = 
# GET UNIQUE VALUES - what are the priorities for instance 
#>> Introduce different tokenization in order to split name and numbers? 

#!!!! HOW MANY THIS AND(OR) THAT COLUMN_NAME ARE THERE? 
sample0 = recombinator('f_select_unique col_1 df_name',
                       ['Which {col_1} are there','What are the {col_1}',
                        'What are {col_1}',
                        'Name the {col_1}','Name {col_1}','Tell me unique {col_1}',
                        'Name unique {col_1}','What are unique {col_1}', 'Name different {col_1}',
                        'What are the different {col_1}','What are different {col_1}'],
                       [column_name('col_1')], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df])
sample00 = recombinator('f_length f_select_unique col_1 df_name',
                       ['How many {col_1}','What is the number of {col_1}','What is number of {col_1}', 'How many different {col_1}',
                        'How many {col_1} are there','How many {col_1} is there', 'Count {col_1}', 'Tell me the number of {col_1}','Tell me number of {col_1}',
                        'Count different {col_1}','How many unique {col_1}', 'Count unique {col_1}','Count the different {col_1}', 'Count the unique {col_1}'],
                       [column_name('col_1')], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df])
# >>>>>>> - Add the unique variation with filters as well
sample1 = recombinator('f_select col_1 f_filter df_name f_eq col_2 var_2',
                       ['Which {col_1} have {var_2} {col_2}','What {col_1} have {var_2} {col_2}',
                        'Which {col_1} have {col_2} equal to {var_2}',
                        'Name the {col_1} where {col_2} is {var_2}', 'Name the {col_1} which have {var_2} {col_2}',
                        'Name the {col_1} with {var_2} {col_2}','Name {col_1} where {col_2} is {var_2}','Name {col_1} with {var_2} {col_2}'],
                       [column_name('col_1'),(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df])
#>------------COMP TESTING ------------ 
sample2 = recombinator('f_select col_1 f_filter df_name* f_eq col_2 var_2',
                       ['Which {col_1} have {var_2} {col_2}*','What {col_1} have {var_2} {col_2}*',
                        'Which {col_1} have {col_2} equal to {var_2}*',
                        'Name the {col_1} where {col_2} is {var_2}*', 'Name the {col_1} which have {var_2} {col_2}*',
                        'Name the {col_1} with {var_2} {col_2}*','Name {col_1} where {col_2} is {var_2}*','Name {col_1} with {var_2} {col_2}*'],
                       [column_name('col_1'),(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df],component_selectors=[(logic_components,'*',5)])
#-------------COMP TESTING -----------> 

sample3 = recombinator('f_length f_filter df_name f_eq col_2 var_2',
                       ['How many {col_1} have {var_2} {col_2}','What is the number of {col_1} having {var_2} {col_2}',
                        'What is the number of {col_1} with {col_2} equal to {var_2}',
                        'How many {col_1} where {col_2} is {var_2}', 'How many {col_1} which have {var_2} {col_2}',
                        'How many {col_1} with {var_2} {col_2}','What is the number of {col_1} where {col_2} is {var_2}'],
                       [column_name('col_1'),(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df])

sample4 = recombinator('f_length f_filter df_name* f_eq col_2 var_2',
                       ['How many {col_1} have {var_2} {col_2}*','What is the number of {col_1} having {var_2} {col_2}*',
                        'What is the number of {col_1} with {col_2} equal to {var_2}*',
                        'How many {col_1} where {col_2} is {var_2}*', 'How many {col_1} which have {var_2} {col_2}*',
                        'How many {col_1} with {var_2} {col_2}*','What is the number of {col_1} where {col_2} is {var_2}*'],
                       [column_name('col_1'),(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df],component_selectors=[(logic_components,'*',5)])


sample5 = recombinator('f_length f_filter df_name f_eq col_2 var_2',
                       ['How many {var_2} {col_2}', 'What is the number of {var_2} {col_2}'],
                       [(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df])


# here the first and second column_names should be the same, we are not adressing this right now
# WHAT IS THE LARGEST PRIORITY - take the largest group
sample6 = recombinator('f_select col_1 f_max col_1 f_filter df_name f_eq col_2 var_2',
                       ['What is the largest {col_1} with {var_2} {col_2}','What is the largest {col_1} having {var_2} {col_2}',
                        'What is the largest {col_1} where {col_2} is equal to {var_2}',
                        'What is the largest {col_1} where {col_2} is {var_2}', 'What is the largest {col_1} which have {var_2} {col_2}',
                        'Name the largest {col_1} with {var_2} {col_2}','Name the largest {col_1} having {var_2} {col_2}',
                        'Name the largest {col_1} where {col_2} is equal to {var_2}',
                        'Name the largest {col_1} where {col_2} is {var_2}', 'Name the largest {col_1} which have {var_2} {col_2}'],
                       [column_name('col_1'),(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df])
sample7 = recombinator('f_select col_1 f_max col_1 f_filter df_name* f_eq col_2 var_2',
                       ['What is the largest {col_1} with {var_2} {col_2}*','What is the largest {col_1} having {var_2} {col_2}*',
                        'What is the largest {col_1} where {col_2} is equal to {var_2}*',
                        'What is the largest {col_1} where {col_2} is {var_2}*', 'What is the largest {col_1} which have {var_2} {col_2}*',
                        'Name the largest {col_1} with {var_2} {col_2}*','Name the largest {col_1} having {var_2} {col_2}*',
                        'Name the largest {col_1} where {col_2} is equal to {var_2}*',
                        'Name the largest {col_1} where {col_2} is {var_2}*', 'Name the largest {col_1} which have {var_2} {col_2}*'],
                       [column_name('col_1'),(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df],component_selectors=[(logic_components,'*',5)])


# note the trick with the 'price token', this column doesn't have to exist in a specific dataset, but we map price to the closest option 
# The words which are not present in the query should be mapped to a word in output vocab, e.g. expensive -> price, sold -> ...
# Consider things like 'least valuable' it implies some knowledge, e.g. it contains information which is not directly stated 
sample9 = recombinator('f_select col_1 f_max price f_filter df_name f_eq col_2 var_2',
                       ['What is the most expensive {col_1} with {var_2} {col_2}','What is the most expensive {col_1} having {var_2} {col_2}',
                        'What is the most expensive {col_1} where {col_2} is equal to {var_2}',
                        'What is the most expensive {col_1} where {col_2} is {var_2}', 'What is the most expensive {col_1} which have {var_2} {col_2}',
                        'Name the most expensive {col_1} with {var_2} {col_2}','Name the most expensive {col_1} where {col_2} is {var_2}',
                        'Name the most expensive {col_1} where {col_2} is equal to {var_2}', 'Name the most expensive {col_1} which have {var_2} {col_2}',
                        'Name the most expensive {col_1} having {var_2} {col_2}'],
                       [column_name('col_1'),(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df])
sample10 = recombinator('f_select col_1 f_max price f_filter df_name* f_eq col_2 var_2',
                       ['What is the most expensive {col_1} with {var_2} {col_2}*','What is the most expensive {col_1} having {var_2} {col_2}*',
                        'What is the most expensive {col_1} where {col_2} is equal to {var_2}*',
                        'What is the most expensive {col_1} where {col_2} is {var_2}*', 'What is the most expensive {col_1} which have {var_2} {col_2}*',
                        'Name the most expensive {col_1} with {var_2} {col_2}*','Name the most expensive {col_1} where {col_2} is {var_2}*',
                        'Name the most expensive {col_1} where {col_2} is equal to {var_2}*', 'Name the most expensive {col_1} which have {var_2} {col_2}*',
                        'Name the most expensive {col_1} having {var_2} {col_2}*'],
                       [column_name('col_1'),(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df],component_selectors=[(logic_components,'*',5)])


sample13 = recombinator('f_select col_1 f_min col_1 f_filter df_name f_eq col_2 var_2',
                       ['What is the smallest {col_1} with {var_2} {col_2}','What is the smallest {col_1} having {var_2} {col_2}',
                        'What is the smallest {col_1} where {col_2} is equal to {var_2}',
                        'What is the smallest {col_1} where {col_2} is {var_2}', 'What is the smallest {col_1} which have {var_2} {col_2}',
                        'Name the smallest {col_1} with {var_2} {col_2}','Name the smallest {col_1} having {var_2} {col_2}',
                        'Name the smallest {col_1} where {col_2} is equal to {var_2}',
                        'Name the smallest {col_1} where {col_2} is {var_2}', 'Name the smallest {col_1} which have {var_2} {col_2}'],
                       [column_name('col_1'),(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df])
sample14 = recombinator('f_select col_1 f_min col_1 f_filter df_name* f_eq col_2 var_2',
                       ['What is the smallest {col_1} with {var_2} {col_2}*','What is the smallest {col_1} having {var_2} {col_2}*',
                        'What is the smallest {col_1} where {col_2} is equal to {var_2}*',
                        'What is the smallest {col_1} where {col_2} is {var_2}*', 'What is the smallest {col_1} which have {var_2} {col_2}*',
                        'Name the smallest {col_1} with {var_2} {col_2}*','Name the smallest {col_1} having {var_2} {col_2}*',
                        'Name the smallest {col_1} where {col_2} is equal to {var_2}*',
                        'Name the smallest {col_1} where {col_2} is {var_2}*', 'Name the smallest {col_1} which have {var_2} {col_2}*'],
                       [column_name('col_1'),(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df],component_selectors=[(logic_components,'*',5)])


# Try out least expensive instead of cheapest as well!  
sample16 = recombinator('f_select col_1 f_min price f_filter df_name f_eq col_2 var_2',
                       ['What is the cheapest {col_1} with {var_2} {col_2}','What is the most cheapest {col_1} having {var_2} {col_2}',
                        'What is the cheapest {col_1} where {col_2} is equal to {var_2}',
                        'What is the cheapest {col_1} where {col_2} is {var_2}', 'What is the cheapest {col_1} which have {var_2} {col_2}',
                        'Name the cheapest {col_1} with {var_2} {col_2}','Name the cheapest {col_1} where {col_2} is {var_2}',
                        'Name the cheapest {col_1} where {col_2} is equal to {var_2}', 'Name the cheapest {col_1} which have {var_2} {col_2}',
                        'Name the cheapest {col_1} having {var_2} {col_2}'],
                       [column_name('col_1'),(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df])
sample17 = recombinator('f_select col_1 f_min price f_filter df_name* f_eq col_2 var_2',
                       ['What is the cheapest {col_1} with {var_2} {col_2}*','What is the most cheapest {col_1} having {var_2} {col_2}*',
                        'What is the cheapest {col_1} where {col_2} is equal to {var_2}*',
                        'What is the cheapest {col_1} where {col_2} is {var_2}*', 'What is the cheapest {col_1} which have {var_2} {col_2}*',
                        'Name the cheapest {col_1} with {var_2} {col_2}*','Name the cheapest {col_1} where {col_2} is {var_2}*',
                        'Name the cheapest {col_1} where {col_2} is equal to {var_2}*', 'Name the cheapest {col_1} which have {var_2} {col_2}*',
                        'Name the cheapest {col_1} having {var_2} {col_2}*'],
                       [column_name('col_1'),(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df],component_selectors=[(logic_components,'*',5)])

# f_select,column_name,f_max,'count',f_count,column_name,f_filter,df_name,f_eq,column_name,var
# [What] [car brand] [had sold] [the most] [[yellow] [cars]]? - here the name of the column 'color' is not mentioned but it is implied by the meaning of word 'yellow' 
# What {col_1} has the most {var_2} {col_2} and/or
# What {col_1} has the largest number of {var_2} {col_2} and/or
# What {col_1} has the most {col_2} being {var_2}
# What {col_1} has the most {col_2} equal to {var_2} / the largest number of
# What {col_1} has the most entries where {col_2} is {var_2} / the largest number of
# What {col_1} has the most entries with {var_2} {col_2}/ the largest number of
#>> add samples with group 
sample19 = recombinator('f_select col_1 f_max count f_count col_1 f_filter df_name f_eq col_2 var_2',
                       ['What {col_1} has the most {var_2} {col_2}','What {col_1} has the largest number of {var_2} {col_2}',
                        'What {col_1} has the most {col_2} being {var_2}', 'What {col_1} has the largest number of {col_2} being {var_2}',
                        'What {col_1} has the most {col_2} equal to {var_2}', 'What {col_1} has the largest number of {col_2} equal to {var_2}',
                        'What {col_1} has the most entries where {col_2} is {var_2}','What {col_1} has the largest number of entries where {col_2} is {var_2}',
                        'What {col_1} has the most entries with {var_2} {col_2}', 'What {col_1} has the largest number of entries with {var_2} {col_2}'],
                       [column_name('col_1'),(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df])
sample20 = recombinator('f_select col_1 f_max count f_count col_1 f_filter df_name* f_eq col_2 var_2',
                       ['What {col_1} has the most {var_2} {col_2}*','What {col_1} has the largest number of {var_2} {col_2}*',
                        'What {col_1} has the most {col_2} being {var_2}*', 'What {col_1} has the largest number of {col_2} being {var_2}*',
                        'What {col_1} has the most {col_2} equal to {var_2}*', 'What {col_1} has the largest number of {col_2} equal to {var_2}*',
                        'What {col_1} has the most entries where {col_2} is {var_2}*','What {col_1} has the largest number of entries where {col_2} is {var_2}*',
                        'What {col_1} has the most entries with {var_2} {col_2}*', 'What {col_1} has the largest number of entries with {var_2} {col_2}*'],
                       [column_name('col_1'),(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df],component_selectors=[(logic_components,'*',5)])


# ----------------------
sample22 = recombinator('f_select col_1 f_min count f_count col_1 f_filter df_name f_eq col_2 var_2',
                       ['What {col_1} has the least {var_2} {col_2}','What {col_1} has the smallest number of {var_2} {col_2}',
                        'What {col_1} has the least {col_2} being {var_2}', 'What {col_1} has the smallest number of {col_2} being {var_2}',
                        'What {col_1} has the least {col_2} equal to {var_2}', 'What {col_1} has the smallest number of {col_2} equal to {var_2}',
                        'What {col_1} has the least entries where {col_2} is {var_2}','What {col_1} has the smallest number of entries where {col_2} is {var_2}',
                        'What {col_1} has the least entries with {var_2} {col_2}', 'What {col_1} has the smallest number of entries with {var_2} {col_2}'],
                       [column_name('col_1'),(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df])
# ----------------------
sample23 = recombinator('f_select col_1 f_min count f_count col_1 f_filter df_name* f_eq col_2 var_2',
                       ['What {col_1} has the least {var_2} {col_2}*','What {col_1} has the smallest number of {var_2} {col_2}*',
                        'What {col_1} has the least {col_2} being {var_2}*', 'What {col_1} has the smallest number of {col_2} being {var_2}*',
                        'What {col_1} has the least {col_2} equal to {var_2}*', 'What {col_1} has the smallest number of {col_2} equal to {var_2}*',
                        'What {col_1} has the least entries where {col_2} is {var_2}*','What {col_1} has the smallest number of entries where {col_2} is {var_2}*',
                        'What {col_1} has the least entries with {var_2} {col_2}*', 'What {col_1} has the smallest number of entries with {var_2} {col_2}*'],
                       [column_name('col_1'),(column_name('col_2'),var('var_2'))], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df],component_selectors=[(logic_components,'*',5)])

#----------------------------------<>--------------------------
sample25 = recombinator('f_select col_1 f_max col_1 df_name',
                       ['What is the largest {col_1}','Name the largest {col_1}','What is the biggest {col_1}','Name the biggest {col_1}','Tell me the largest {col_1}','Tell me the biggest {col_1}',
                        'What {col_1} is the largest','What {col_1} is the biggest'], #replicate this for the cases with filters 
                       [column_name('col_1')], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df])
sample26 = recombinator('f_select col_1 f_min col_1 df_name',
                       ['What is the smallest {col_1}','Name the smallest {col_1}','Tell me the smallest {col_1}', 'What {col_1} is the smallest'],
                       [column_name('col_1')], #objects in a tuple are linked 
                       [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df])

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> new samples:
#sample27 =  Group by and sum X, Sum X by Y
#sample28 = Group by and Average (take average, get average)
# Group by A,B,C,D and E and show me the group sizes/sizes, sizes of the groups
####
#>>> Save as a parameter to use later, propose to save files to be able to do some data science from mobile - chat window
# 1. Just group by -> bot proposed the options of what to do next
# 2. Group by + size 
# 3. Group by + sum(mean/average etc...) a,b,c... (or get for all numerical columns)
#>>>> Important! do we want to detect the dataset name ?!!!

sample27 = recombinator('f_group df_name* col_1', #at least one grouping column should be mentioned, otherwise error -> **do we want to detect error cases separately?**
                        ['Group by {col_1}*'],
                        [column_name('col_1')],
                        [zoo_df,zoo3_df,avocado_df,cars_df,wildfires_df,bike_df],
                        component_selectors=[(and_col_selector,'*',5)]) 

# Size/ length of column_name group? poisonous group (non-poisonous != true) 
#sum a and b
#there are som assumptions regarding the order which are printed out to the user 
#sum a and b devided by c
#sum a and b and devide by c 
#multiply a and b 
#devide a by b, devide a and b 
#deduct a from b, subtract a from b, subtract a and b from c, deduct a and b from c
#Count each type of, How many items of each type 
# Sort 
# filter large/smaller (number or another column)
# contains
# Plot
# Merge files
# Ensure Filtering is working
# Delete column/row
# Raname column in excel 
# add a new column/row to table 
# All should run super fast!! - only vectorized code
# add education al aspects to teach Python/DS
# Save dialog as module and deploy

#> 'and', 'or' structures of the arbitrary length
#> Complex recombinations
#> Approximate column search
#> Execution based on the column type
#> Inputting column metadata to the model, What animals are poisonous, What animals (animal_name) are poisonous -> spider =1 amd poisonous = 1
# -- add a new sample 
#> When prices in New York were the best (When -> year/time, New York -> state), if New York is not present -> we give 0 results,
#> if there is a spiders column -> pick it and filter by poisonous
#'f_select col_1 f_filter df_name f_eq col_2 true, ['What {col_1} are {col_2}' #replace true with 1 where applicable

#! 'f_select col_1 f_filter df_name f_and f_eq col_1 var_2 f_eq col_2 true, ['What {var_2} are {col_2}' - What spiders (spider=1, or animal_type) are poisonous (non-poisonous: False)
#! Filter not-equal to  
# What animals (animal_name) are poisonous
# column types: bin, num, text (cat)
# we need column names and types in the input
# if no specific column specified -> select all? or pick the most appropriate? 
#'f_select col_1 f_filter df_name f_and f_eq col_1 var_2 f_eq col_2 true, ['What are {col_2} {var_2}' What are poisonous (and flying) spiders? 
#'f_select col_1 f_filter df_name f_and f_eq col_1 var_2 f_eq col_2 true, ['What are {col_2} {col_1}'
#> Two stage model where we may check column metadata in between (type and contents):
#> Run the entity recognition -> anylize the entities -> input those to the next level, if not present do classification? 
#> alternative input column names and do 1/0 classification in order to get metadata 
# Transformer => input econder - column/var names -> input decoder - all column names -> output classification by col_1, col_2 etc. 
# When column is not matched directly we get the closest column and assume it is a variable 
# Do not map column names yet, just add 'What animal_name are poisonous' = 'f_select col_1 f_filter df_name f_eq col_2 true,

# variable name is created automatically, give a way to name it like 'as *name*'
# take real life samples >>> 
# >>> add the ratio samples as well 
#Sample:
# Difference (substraction)
# Group and sum a column 
# Group and sum / group and size
# 

In [ ]:
test = sample27.generate(1)

In [ ]:
test.iloc[0].rep #.text; .ner

'f_group df_name f_and col_2 f_and col_3 f_and col_4 f_and col_5 col_1'

In [ ]:
def run_recombinators(n,recombinator_list):
  output_dfs = []
  for recombinator in recombinator_list:
    output_dfs.append(recombinator.generate(n))
  return pd.concat(output_dfs, ignore_index=True)

In [ ]:
df = run_recombinators(3,[sample0,sample00,sample1,sample2,sample3,sample4,sample5,sample6,sample7,sample9,sample10,sample13,sample14,sample16,sample17,sample19,sample20,sample22,sample23,sample25,sample26])

30 for samples_l, 1 for valiation

In [ ]:
# shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True).reset_index(drop=True)

In [ ]:
df.to_csv('/content/drive/My Drive/Datasets/validate_ner_ref.csv',index=False)

In [ ]:
len(df)

3402

In [ ]:
tokenizer.tokenize('How many Unnamed: 0 have Chicago region')

['how', 'many', 'unnamed', ':', '0', 'have', 'chicago', 'region']

In [ ]:
df

,text,rep,ner
0,Which toothed are there,f_select_unique col_1 df_name,O col_1 col_1 O O
1,Which fins are there,f_select_unique col_1 df_name,O col_1 O O
2,Which catsize are there,f_select_unique col_1 df_name,O col_1 col_1 O O
3,Which predator are there,f_select_unique col_1 df_name,O col_1 O O
4,Which backbone are there,f_select_unique col_1 df_name,O col_1 O O
...,...,...,...
17005,What feature_6 is the smallest,f_select col_1 f_min col_1 df_name,O col_1 col_1 col_1 O O O
17006,What engine_has_gas is the smallest,f_select col_1 f_min col_1 df_name,O col_1 col_1 col_1 col_1 col_1 O O O
17007,What year_produced is the smallest,f_select col_1 f_min col_1 df_name,O col_1 col_1 col_1 O O O
17008,What location_region is the smallest,f_select col_1 f_min col_1 df_name,O col_1 col_1 col_1 O O O


In [ ]:
pd.concat([test_pd,test_pd], ignore_index=True)

,text,rep
0,Which aquatic have 0 catsize,"f_select,column_name,f_filter,df_name,f_eq,col..."
1,Which domestic have 0 milk,"f_select,column_name,f_filter,df_name,f_eq,col..."
2,Which feathers have 3 class_type,"f_select,column_name,f_filter,df_name,f_eq,col..."
3,Which Large Bags have 1.16 AveragePrice,"f_select,column_name,f_filter,df_name,f_eq,col..."
4,Which year have 2016-01-31 Date,"f_select,column_name,f_filter,df_name,f_eq,col..."
...,...,...
85,Name the Total Bags which have 0.0 XLarge Bags,"f_select,column_name,f_filter,df_name,f_eq,col..."
86,Name the Unnamed: 0 which have 39.38 Large Bags,"f_select,column_name,f_filter,df_name,f_eq,col..."
87,Name the engine_has_gas which have False featu...,"f_select,column_name,f_filter,df_name,f_eq,col..."
88,Name the year_produced which have 1 up_counter,"f_select,column_name,f_filter,df_name,f_eq,col..."


In [ ]:
['hey','there','hey'].index('hey')

0

In [ ]:
df

,text,rep
0,Which predator have 0 milk,f_select column_name 1 1 f_filter df_name f_eq...
1,Which Large Bags have 74597.93 4225,f_select column_name 1 2 f_filter df_name f_eq...
2,Which manufacturer_name have gasoline engine_type,f_select column_name 1 1 f_filter df_name f_eq...
3,What milk have 1 aquatic,f_select column_name 1 1 f_filter df_name f_eq...
4,What Small Bags have BaltimoreWashington region,f_select column_name 1 2 f_filter df_name f_eq...
...,...,...
670,What 4046 has the least entries with 10769.2 T...,f_select column_name 1 1 f_min count f_count c...
671,What duration_listed has the least entries wit...,f_select column_name 1 1 f_min count f_count c...
672,What breathes has the smallest number of entri...,f_select column_name 1 1 f_min count f_count c...
673,What AveragePrice has the smallest number of e...,f_select column_name 1 1 f_min count f_count c...


In [ ]:
import numpy as np

In [ ]:
tensor_01 = np.random.randint(0,2, (1000,1,100)) #Alternative could be a matrix where each line is multiplied by a different matrix from tensor_02
tensor_02 = np.random.randint(0,2, (1000,100,50))
# Goal matrix 1000*50

In [ ]:
np.einsum('ijk,ikv->iv', tensor_01,tensor_02)

array([[29, 24, 24, ..., 31, 28, 21],
       [19, 25, 23, ..., 20, 22, 24],
       [23, 25, 22, ..., 20, 30, 29],
       ...,
       [23, 20, 25, ..., 16, 24, 22],
       [28, 26, 27, ..., 21, 28, 25],
       [24, 24, 20, ..., 19, 27, 20]])